In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os


/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [19]:
def imgFilt(img,filtratio):
    ratio = filtratio*255
    filtered_img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    grimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2GRAY)
    
    #hsvimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2HSV)
        
    lightMask = np.asarray(grimg)
    Mask_indices = lightMask > ratio
    filtered_img[Mask_indices] =255
    return filtered_img,Mask_indices

In [20]:
def imgHsvFilt(img):

    filtered_img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    #grimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2GRAY)
    
    hsvimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2HSV)
        
    h,s,vMask = cv2.split(hsvimg)
    Mask_indices = vMask > 150
    filtered_img[Mask_indices] =255
    #return filtered_img
    return filtered_img,Mask_indices

In [21]:
def detectDark(img,rat):
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    hei,wid,chan = hsv.shape
    ratio = np.sum(v)/(hei*wid*255)
    if(ratio < rat):
        return True
    else:
        return False

In [22]:
def color_detection(frame_in,ratio):
    # Take each frame
    #frame = cv2.imread(image_name)
        
        
        if(detectDark(frame_in,ratio)):
            roiWid = 10
            roiEdg = 8
        else:
            roiWid = 4
            roiEdg = 2

        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / roiWid)
        roiWidth = roiX * roiEdg
        roiY = int(src_height / roiWid)
        roiHeight = roiY * roiEdg
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        ##below is changed
#         if(filt):
#             frame,lightMask = imgFilt(frame)
#             src_height, src_width, src_channels = frame.shape
#             blockedPxl = sum(np.count_nonzero(e) for e in lightMask)
#             max_value = ((src_height*src_width)-blockedPxl)*255
#         #print src_height,src_width,max_value
#         else:
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255
        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        #hsv= cv2.cvtColor(filtered_img, cv2.COLOR_BGR2HSV)
        # detect red
        lower = np.array([136, 73, 71])
        upper = np.array([186, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        outmask = mask
        redVal = float(mask.sum()) / float(max_value)
        if redVal > 0.2:
            red="true"
        else:
            red="false"

        # detect yellow
        lower = np.array([10, 50, 60])
        upper = np.array([37, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        yellowVal = float(mask.sum()) / float(max_value)
        if yellowVal > 0.15:
            yellow="true"
        else:
            yellow="false"

        # detect blue
        lower = np.array([85, 50, 60])
        upper = np.array([140, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        blueVal = float(mask.sum()) / float(max_value)
        if blueVal > 0.35:
            blue="true"
        else:
            blue="false"

        # detect green (gray)
        lower_green = np.array([42, 26, 41])
        upper_green = np.array([99, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        greenVal = float(mask.sum()) / float(max_value)
        if greenVal > 0.01:
            green="true"
        else:
            green="false"

        #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # detect white
        lower = np.array([0, 0, 140])
        upper = np.array([179, 60, 256])
        mask = cv2.inRange(hsv, lower, upper)
#         if(filt):
#             whiteVal = float(mask.sum()-blockedPxl*255) / float(max_value)
#         else:
        whiteVal = float(mask.sum()) / float(max_value)
        
        if whiteVal > 0.5:
            white="true"
        else:
            white="false"

        # detect black
        lower_black = np.array([0,0,0])
        upper_black= np.array([179,255,60])
        mask = cv2.inRange(hsv, lower_black, upper_black)
        blackVal = float(mask.sum()) / float(max_value)
        if blackVal > 0.01:
            black="true"
        else:
            black="false"

        return [red, yellow, blue, green, white, black]
        #return redVal, yellowVal, blueVal, greenVal, whiteVal, blackVal
        #return outmask

In [23]:
ROOT = '/Users/jeffshih/Desktop/work/color-detection/exp/img/Car_img'
os.chdir(ROOT)
file_list = []
label_list = []
for root,subdir,files in os.walk(ROOT):
    if(len(root.split("/"))>9):
        if (root.split("/")[9].split("_")[0]=='21'):
            for fname in files:
                if(len(os.path.splitext(root)[0].split("_"))==3):
                    file_list.append(os.path.join(root,fname))
                    label_list.append(os.path.splitext(root)[0].split("_")[2])
                #print fname

test_dark_file = []
test_dark_label = []
os.chdir(ROOT)
for root,subdir,files in os.walk(ROOT):
    #print root
    if(len(root.split("/"))>9):
        if (root.split("/")[9].split("_")[0]=='12' or root.split("/")[9].split("_")[0]=='21'):
            for fname in files:
                if(len(os.path.splitext(root)[0].split("_"))==3):
                    test_dark_file.append(os.path.join(root,fname))
                    #test_dark_label.append(os.path.splitext(root)[0].split("_")[2])
                    #print os.path.splitext(root)[0].split("_")[1].split("/")[1]
                    test_dark_label.append(os.path.splitext(root)[0].split("_")[1].split("/")[1])

In [24]:
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

In [10]:
img_cnt = len(test_dark_file)
Tpos = 0
Tneg = 0
Fpos = 0
Fneg = 0

for rat in xrange(70,80,1):
    rat = float(rat)/100.0
    print rat
    for i in range(0,img_cnt):
        #print test_dark_label[i]
        img = cv2.imread(test_dark_file[i])
        if(detectDark(img,rat) and test_dark_label[i]=='12'):
            Fpos+=1
        if(detectDark(img,rat) and test_dark_label[i]=='21'):
            Tpos+=1
        if(detectDark(img,rat) == False and test_dark_label[i]=='12'):
            Tneg+=1
        if(detectDark(img,rat) == False and test_dark_label[i]=='21'):
            Fneg+=1

    print "Detect Dark Test"
    #print Tpos,Tneg,Fpos,Fneg
    print "False Pos : ",float(Fpos)/float(Tpos+Fpos)
    print "False Neg : ",float(Fneg)/float(Tneg+Fneg)

0.7
Detect Dark Test
False Pos :  0.61648177496
False Neg :  0.0833333333333
0.71
Detect Dark Test
False Pos :  0.620106482931
False Neg :  0.0843373493976
0.72


KeyboardInterrupt: 

In [13]:
color_label = ["red","yellow","blue","green","white","black"]
true_label = [0,0,0,0,0,0]
true_pos_pred_b = [0,0,0,0,0,0]
true_pos_pred_a = [0,0,0,0,0,0]
false_pos_pred_b =[0,0,0,0,0,0]
false_pos_pred_a =[0,0,0,0,0,0]
false_neg_pred_b = [0,0,0,0,0,0]
false_neg_pred_a = [0,0,0,0,0,0]
correct_pred_b = [0,0,0,0,0,0]
correct_pred_a = [0,0,0,0,0,0]
true_label = [0,0,0,0,0,0]
total_img_cnt = len(file_list)
print 'total img cnt :',total_img_cnt 
#print ''len(label_list)
 

for i in range(0,total_img_cnt):
    if (np.random.rand()<1.1):
        pred_label_idx = color_label.index(label_list[i])
        true_label[pred_label_idx]+=1
        filename = file_list[i]
        img = cv2.imread(filename)
        label = label_list[i]
        pred_label_before_filt = color_detection(img)
        filt_img,mask_idx= imgFilt(img)
        pred_label_after_filt = color_detection(filt_img)
        pred_idx_b = list_duplicates_of(pred_label_before_filt,'true')
        pred_idx_a = list_duplicates_of(pred_label_after_filt,'true')
        neg_idx_b = list_duplicates_of(pred_label_before_filt,'false')
        neg_idx_a = list_duplicates_of(pred_label_after_filt,'false')
        for idx in pred_idx_b:
            if(label==color_label[idx]):
                true_pos_pred_b[idx] +=1
            else:
                false_pos_pred_b[idx] +=1
        for idx in neg_idx_b:
            if(label==color_label[idx]):
                false_neg_pred_b[idx] +=1

        for idx in pred_idx_a:
            #pos_pred_after_filt[idx]+=1
            if(label==color_label[idx]):
                true_pos_pred_a[idx] +=1
            else:
                false_pos_pred_a[idx] +=1

        for idx in neg_idx_a:
            if(label==color_label[idx]):
                false_neg_pred_a[idx] +=1

sampled_cnt = sum(true_label)
print 'sampled img cnt :', sampled_cnt 
print "finished"

total img cnt : 1227
sampled img cnt : 1227
finished


In [30]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import time
from pprint import pprint
def objective(args):
    sample = 0.7
    darkratio, filtratio= args
    color_label = ["red","yellow","blue","green","white","black"]
    true_label = [0,0,0,0,0,0]
    true_pos_pred_b = [0,0,0,0,0,0]
    true_pos_pred_a = [0,0,0,0,0,0]
    false_pos_pred_b =[0,0,0,0,0,0]
    false_pos_pred_a =[0,0,0,0,0,0]
    false_neg_pred_b = [0,0,0,0,0,0]
    false_neg_pred_a = [0,0,0,0,0,0]
    correct_pred_b = [0,0,0,0,0,0]
    correct_pred_a = [0,0,0,0,0,0]
    true_label = [0,0,0,0,0,0]
    total_img_cnt = len(file_list)
    #print ''len(label_list)


    for i in range(0,total_img_cnt):
        if (np.random.rand()<sample):
            pred_label_idx = color_label.index(label_list[i])
            true_label[pred_label_idx]+=1
            filename = file_list[i]
            img = cv2.imread(filename)
            label = label_list[i]
            pred_label_before_filt = color_detection(img,darkratio)
            filt_img,mask_idx= imgFilt(img,filtratio)
            pred_label_after_filt = color_detection(filt_img,darkratio)
            pred_idx_b = list_duplicates_of(pred_label_before_filt,'true')
            pred_idx_a = list_duplicates_of(pred_label_after_filt,'true')
            neg_idx_b = list_duplicates_of(pred_label_before_filt,'false')
            neg_idx_a = list_duplicates_of(pred_label_after_filt,'false')
            for idx in pred_idx_b:
                if(label==color_label[idx]):
                    true_pos_pred_b[idx] +=1
                else:
                    false_pos_pred_b[idx] +=1
            for idx in neg_idx_b:
                if(label==color_label[idx]):
                    false_neg_pred_b[idx] +=1

            for idx in pred_idx_a:
                #pos_pred_after_filt[idx]+=1
                if(label==color_label[idx]):
                    true_pos_pred_a[idx] +=1
                else:
                    false_pos_pred_a[idx] +=1

            for idx in neg_idx_a:
                if(label==color_label[idx]):
                    false_neg_pred_a[idx] +=1

    sampled_cnt = sum(true_label)
    fp = 0
    fn = 0
    for i in range(6):
        curCol = color_label[i]
        #TrueNeg = 1227-true_pos_pred_b[i]-false_pos_pred_b[i]-false_neg_pred_b[i]
        fp += float(false_pos_pred_b[i])/float(true_pos_pred_b[i]+false_pos_pred_b[i])
        fn += float(false_neg_pred_b[i])/float(true_label[i])
    return {
        'loss':fp + fn,
        'status': STATUS_OK,
        'eval_time':time.time()
    }

In [31]:
space = [hp.uniform('filtratio',0.3,0.7),hp.uniform('darkratio',0.3,0.7),
        ]
print '\n'
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=500,trials= trials)
pprint(best)
print '\n'
pprint(trials.losses())

KeyboardInterrupt: 

In [14]:



print 'true label :',true_label
print 'true postive predict before filter :',true_pos_pred_b
print 'false postive predict before filter :',false_pos_pred_b
#print 'sum of true class before filt:',sum(true_pred_b)
print 'false neg predice before filter :',false_neg_pred_b
for i in range(6):
    curCol = color_label[i]
    #TrueNeg = 1227-true_pos_pred_b[i]-false_pos_pred_b[i]-false_neg_pred_b[i]
    print curCol , " before filtering, false pos rate is : ", float(false_pos_pred_b[i])/float(true_pos_pred_b[i]+false_pos_pred_b[i])
    print curCol , " before filtering, false neg rate is : ", float(false_neg_pred_b[i])/float(true_label[i])
    
print "========after filtered========"

print 'true postive predict after filter :',true_pos_pred_a
print 'false postive predict after filter :',false_pos_pred_a
#print 'sum of true class before filt:',sum(true_pred_b)
print 'false neg predice after filter :',false_neg_pred_a
for i in range(6):
    curCol = color_label[i]
    TrueNeg = 1227-true_pos_pred_a[i]-false_pos_pred_a[i]-false_neg_pred_a[i]
    print curCol , " after filtering, false pos rate is : ", float(false_pos_pred_a[i])/float(true_pos_pred_a[i]+false_pos_pred_a[i])
    print curCol , " after filtering, false neg rate is : ", float(false_neg_pred_a[i])/float(true_label[i])

true label : [115, 171, 101, 17, 503, 320]
true postive predict before filter : [81, 164, 59, 17, 297, 309]
false postive predict before filter : [49, 47, 49, 738, 11, 715]
false neg predice before filter : [34, 7, 42, 0, 206, 11]
red  before filtering, false pos rate is :  0.376923076923
red  before filtering, false neg rate is :  0.295652173913
yellow  before filtering, false pos rate is :  0.222748815166
yellow  before filtering, false neg rate is :  0.0409356725146
blue  before filtering, false pos rate is :  0.453703703704
blue  before filtering, false neg rate is :  0.415841584158
green  before filtering, false pos rate is :  0.977483443709
green  before filtering, false neg rate is :  0.0
white  before filtering, false pos rate is :  0.0357142857143
white  before filtering, false neg rate is :  0.409542743539
black  before filtering, false pos rate is :  0.6982421875
black  before filtering, false neg rate is :  0.034375
========after filtered========
true postive predict after 

In [ ]:
filt = False

img = cv2.imread('./img/redlight6.jpg')#
#img = cv2.imread('./whitebalanced.png')
#img2 = cv2.imread('./redlight6.jpg')
height,width,chann = img.shape


print "red,    yellow,   blue,    green,    white,    black"
print color_detection(img)

filtered_img,mask = imgFilt(img)

filt = True
#filtered_img= imgFilt(img)
print color_detection(img)
grimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


if(detectDark(img)):
    roiWid = 10
    roiEdg = 8
else:
    roiWid = 4
    roiEdg = 2
        
src_height, src_width, src_channels = img.shape
roiX = int(src_width / roiWid)
roiWidth = roiX * roiEdg
roiY = int(src_height / roiWid)
roiHeight = roiY * roiEdg
img = img[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

#img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
filtered_img = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2RGB)

#dst = cv2.imread('./red2.jpg')
plt.figure(figsize=(20,10))
roiimg = img[roiY:roiY+roiHeight, roiX:roiX+roiWidth]
plt.subplot(221),plt.imshow(img)#
#plt.subplot(221),plt.imshow(img2)
plt.subplot(222),plt.imshow(filtered_img)
plt.subplot(223),plt.imshow(grimg ,cmap='gray')
#plt.subplot(223),plt.imshow(redmask)
img = filtered_img[roiY:roiY+roiHeight, roiX:roiX+roiWidth]
plt.subplot(224),plt.imshow(img)
plt.show()


In [ ]:
img = cv2.imread('./redlight6.jpg')
height,width,chann = img.shape
#img = img[325:450,1040:width]
pxls = height*width

hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
h,s,v = cv2.split(hsv)

hist = cv2.calcHist(h,[0],None,[256],[0,256])

b,g,r = cv2.split(img)

average_color = [np.average(b),np.average(g),np.average(r)]

print average_color, hist.argmax()



In [ ]:
img = cv2.imread('./redlight8.jpg')
#img = cv2.imread('./whitebalanced.png')
#img2 = cv2.imread('./redlight6.jpg')
height,width,chann = img.shape


print "red,    yellow,   blue,    green,    white,    black"
print color_detection(img)


filtered_img= imgFilt(img)
filtered_img = cv2.cvtColor(filtered_img,cv2.COLOR_HSV2B)

print color_detection(img)
grimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)



src_height, src_width, src_channels = img.shape
roiX = int(src_width / 10)
roiWidth = roiX * 8
roiY = int(src_height / 10)
roiHeight = roiY * 8

img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

#img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
filtered_img = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2RGB)

#dst = cv2.imread('./red2.jpg')
plt.figure(figsize=(20,10))
roiimg = img[roiY:roiY+roiHeight, roiX:roiX+roiWidth]
plt.subplot(221),plt.imshow(img)
#plt.subplot(221),plt.imshow(img2)
plt.subplot(222),plt.imshow(filtered_img)
plt.subplot(223),plt.imshow(grimg ,cmap='gray')
#plt.subplot(223),plt.imshow(redmask)
img = filtered_img[roiY:roiY+roiHeight, roiX:roiX+roiWidth]
plt.subplot(224),plt.imshow(img)
plt.show()



In [ ]:
def processcal(nscale,maxscal):
    a = []
    size_step = maxscal/nscale
    for i in range(0,nscale):
         a.append(15 + i*size_step)
    return a

def gaussian(r,c,sigma):
    x_in = range(-(r-1)/2,r/2)
    y_in = range(-(c-1)/2,c/2)
    [x,y] = np.meshgrid(x_in,y_in)
    gauss = np.exp(-(x*x+y*y)/(2*sigma*sigma))
    gauss = gauss/sum(gauss)   
    return gauss 

def msrcr(img,d,nscale,maxscal):
    ssr = 0
    height,width,chan = img.shape
    rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    x,y,z = cv2.split(rgb)
    r = height*width
    c = r
    g = x+y+z
    
